In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import concurrent.futures
import shutil
from pathlib import Path
import numpy as np
import thor.data as data
import thor.parallel as parallel
import thor.visualize as visualize
import thor.analyze as analyze
import thor.option as option
import thor.grid as grid

notebook_name = "parallel_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parallel

In [7]:
 # Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T12:00"
end = "2005-11-13T20:00"

intervals = parallel.get_time_intervals(start, end)
output_parent = base_local / "runs/cpol_demo_parallel"
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

 # Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)

# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)

data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

data.dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

altitude = list(np.arange(0, 20e3 + 500, 500))
altitude = [float(alt) for alt in altitude]
grid_options = grid.create_options(name="geographic", altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.check_options(track_options)
option.save_track_options(track_options, options_directory)
visualize_options = None

2024-10-10 09:38:00,778 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/options/data.yml
2024-10-10 09:38:00,794 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-10 09:38:00,795 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-10-10 09:38:00,796 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/options/grid.yml
2024-10-10 09:38:00,803 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/options/track.yml


In [3]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = []
    for i, time_interval in enumerate(intervals):
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.copy(), visualize_options] 
        args += [output_parent, "gridrad"]
        futures.append(executor.submit(parallel.track_interval, *args))
    parallel.check_futures(futures)

2024-10-10 09:37:48,642 - thor.parallel - ERROR - Generated an exception: track_interval() missing 1 required positional argument: 'dataset_name'
2024-10-10 09:37:48,651 - thor.parallel - ERROR - Generated an exception: track_interval() missing 1 required positional argument: 'dataset_name'
2024-10-10 09:37:48,653 - thor.parallel - ERROR - Generated an exception: track_interval() missing 1 required positional argument: 'dataset_name'
2024-10-10 09:37:48,655 - thor.parallel - ERROR - Generated an exception: track_interval() missing 1 required positional argument: 'dataset_name'
2024-10-10 09:37:48,656 - thor.parallel - ERROR - Generated an exception: track_interval() missing 1 required positional argument: 'dataset_name'
2024-10-10 09:37:48,657 - thor.parallel - ERROR - Generated an exception: track_interval() missing 1 required positional argument: 'dataset_name'
2024-10-10 09:37:48,658 - thor.parallel - ERROR - Generated an exception: track_interval() missing 1 required positional arg

In [4]:
parallel.stitch_run(output_parent, intervals)

2024-10-10 09:37:48,750 - thor.parallel - INFO - Stitching all attribute, mask and record files.


KeyError: 'track'

# Analyze

In [5]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-10 09:37:50,643 - thor.attribute.utils - WARNING - No metadata file found for /home/ewan/THOR_output/runs/cpol_demo_parallel/attributes/mcs/era5_pl/profile.csv.
2024-10-10 09:37:50,644 - thor.attribute.utils - WARNING - No metadata; loading entire dataframe and data types not enforced.


FileNotFoundError: [Errno 2] No such file or directory: '/home/ewan/THOR_output/runs/cpol_demo_parallel/attributes/mcs/era5_pl/profile.csv'

In [2]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation"
)
start_time = np.datetime64("2005-11-13T12:00")
end_time = np.datetime64("2005-11-13T22:00")
visualize.attribute.mcs_series(
    output_parent, start_time, end_time, figure_options, parallel_figure=True
)

NameError: name 'visualize' is not defined